In [ ]:
#this script should grab the latest date that is already included in each table, then add in the missing dates
#until the date is today-3.  It should do it for all search types, and eventually for the more accurate count data
import glob
from aux_scripts.GSC_API_extract import granular_extract
from aux_scripts.insertion_csv_to_db import get_files, insert_files
from aux_scripts.authorize_creds import authorize_creds
from aux_scripts.low_performance_report import low_performers_report
######################storage variables for database and desktop#######################################################
#database variables
#The searchType you input into a table should match the table name
DATABASE_NAME=''
PASSWORD =''
USER = ''
HOST = ''
#I'm not actually sure if I'm using this variable anywhere.
TABLE_NAMES=['web','image','video'] #tables where data is stored in db
db_dict = {'USER':USER,'PASSWORD':PASSWORD,'HOST':HOST,'DATABASE_NAME':DATABASE_NAME,'TABLE_NAMES':TABLE_NAMES}

#desktop variables
#STORAGE_FOLDER is name of of the folder containing the 3 search type folders where csv data are currently stores
STORAGE_FOLDER = '' 

#output folder will usually be same as storage folder, but you can store generated files in a different folder if you want
OUTPUT_FOLDER = STORAGE_FOLDER
#######################################################################################################

#GSC API data #########################################################################################
# credentials will not be changed by default in this file.  If you need to change them, call granular_extract with
# the optional parameter cred_dict of the form {'creds':CLIENT_SECRETS_PATH,'authorized_creds_file':AUTHORIZED_CREDS_FILE}

SC_SITE = '' #The site name you need to use to call GSC API using given credentials CREDS, below

#the earliest date you'll draw from.  My script won't let you query for a date if it's already contained in the 
# storage_folder/search_type/ folder, though (these are parameters for aux_scripts.GSC_API_extract.granular_extract)
NUM_DAYS = 485              # Number of Days, Months to Extract with api, at maximum.  496 is maximum, 3 is min
#warning: NUM_DAYS only refers to api actions, not inserting files into db
SEARCH_TYPES = ['web']#,'image','video'] #list of the table names and the corresponding GSC search types.  Names are the same
gsc_dict={'sc_site':SC_SITE, 'num_days':NUM_DAYS,
          'search_types':SEARCH_TYPES, 'storage_folder':STORAGE_FOLDER,'output_folder': OUTPUT_FOLDER}

#################################variable definition complete#################################################


In [ ]:
#NOW we generate a report detailing low performing, high impression pages and their queries
#might want to update database first, because this only examines things already in it!!!
report_table_names=['acc_web','acc_image','acc_video'] #tables where data is stored in db
report_db_dict = db_dict.copy()
report_db_dict['TABLE_NAMES']=report_table_names
low_performers_report(report_db_dict)

In [ ]:
#inserts things into the database
granular_extract(gsc_dict, acc = False, to_save = True) 
#acc=True if you don't want query data
#to_save = false if you don't want more than one day's worth of data
#this script doesn't WORK! IF ITS THE ACC DATA!

In [ ]:
#inserts things into the database
granular_extract(gsc_dict, acc = False, to_save = True) 
#acc=True if you don't want query data
#to_save = false if you don't want more than one day's worth of data
#this script doesn't WORK! IF ITS THE ACC DATA!


do_date_check = True
print('do_date_check is ',do_date_check)
restrict_to_num_days = True #use if you only want to insert the latest few files into db

for i in range(len(gsc_dict['search_types'])):
    #WARNING: TABLE NAMES and SEARCH TYPES should correspond, by index
    #inserts data from the 'table' subfolder of granular/ into the table 'table' in the database 
    
    #first, get file names
    files = get_files(STORAGE_FOLDER, gsc_dict['search_types'][i])

    if restrict_to_num_days == True:
        
        insert_files(files[-NUM_DAYS:], db_dict, db_dict['TABLE_NAMES'][i], do_date_check)
    else:
        insert_files(files, db_dict, db_dict['TABLE_NAMES'][i], do_date_check)  
        
#NOW we generate a report detailing low performing, high impression pages and their queries
report_table_names=['acc_web','acc_image','acc_video'] #tables where data is stored in db
report_db_dict = db_dict.copy()
report_db_dict['TABLE NAMES']=report_table_names
low_performers_report(report_db_dict)
